In [ ]:
target_adresses = ["d8:47:32:eb:6c:38",
                    "50:c7:bf:19:e6:4d",
                    "18:28:61:3d:94:7a"]

In [ ]:
expname   = "exp006_20250526_sna_kadirerdem"

In [ ]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_CNNv1
from algorithms import RssPosAlgo_NearestNeighbour
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch

In [ ]:
datajsonpath = "../experiments/"+expname+"/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=target_adresses, snap250ms=False)

epochs           = 101
batch_size       = 64
train_test_split = 0.8
kernel_sizes     = [13, 9, 5, 3]  # Changed from single kernelsize to list
window_size      = 50

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = batch_size, window_size=window_size,
                                                                     train_test_split = train_test_split,
                                                                     cnn_data=True, cnn_kernel_sizes=kernel_sizes)

In [ ]:
cnn6 = RssPosAlgo_NeuralNet_CNNv1([8, 16, 8, 2], kernel_sizes)
cnn6.load_state_dict(torch.load('savedmodels/dev011_exp006_cnn.pth'));
cnn6.eval();

cnn7 = RssPosAlgo_NeuralNet_CNNv1([8, 16, 8, 2], kernel_sizes)
cnn7.load_state_dict(torch.load('savedmodels/dev012_exp007_cnn.pth'));
cnn7.eval();

In [ ]:
error = 0  # Initialize the error accumulator
for test_inputs, test_labels in test_loader:
    loc_pred_cnn6 = cnn6(test_inputs)
    loc_pred_cnn6_np = loc_pred_cnn6.detach().numpy()  # Predicted values
    loc_pred_cnn7 = cnn7(test_inputs)
    loc_pred_cnn7_np = loc_pred_cnn7.detach().numpy()  # Predicted values
    loc_pred_cnn_np = (loc_pred_cnn6_np + loc_pred_cnn7_np)/2 
    yts_batch_np = test_labels.detach().numpy()  # True values
    diff = yts_batch_np - loc_pred_cnn_np
    error1 = 0
    for i in range(diff.shape[-1]):
        error1 += np.linalg.norm(diff[:,:,i])
    error += error1/diff.shape[-1]

print("Mean error:", error/len(test_loader))

In [ ]:
num_clusterss = [3,4,5,7,9,11,13]
for num_clusters in num_clusterss:
    print("-"*20)
    print("# of clusters:", num_clusters)
    db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=num_clusters)
    meanerror_nene_interp = 0;
    meanerror_nene        = 0;
    for test_idx, x_test_sample in enumerate(xts): 
        loc_pred_nene          = RssPosAlgo_NearestNeighbour(x_test_sample, db_kmeans)
        loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
        meanerror_nene        += (yts[test_idx].numpy() - loc_pred_nene)**2
        meanerror_nene_interp += np.linalg.norm((yts[test_idx].numpy() - loc_pred_nene_interp))
    print("NeNe       :", np.linalg.norm(meanerror_nene/(test_idx+1)))
    print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))